In [1]:
import mne
import sqlite3
%pylab qt

mne.set_log_level('WARNING')

Populating the interactive namespace from numpy and matplotlib


In [2]:
#raw = mne.io.read_raw_brainvision('/Volumes/Store/Avner/POUYAN-BRAIN/pouyan.vhdr')
#raw = mne.io.read_raw_brainvision('/home/avnerus/Code/eegtest/EEG/test2.vhdr')
raw = mne.io.read_raw_eeglab('/home/avnerus/Code/eegtest/MATTI/matti_rs_cl_fl.set')
#raw = mne.io.read_raw_brainvision('/Volumes/Store/Avner/MATTI-BRAIN/matti.vhdr')

print(raw.info)

The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
Events like the following will be dropped entirely: [u'boundary', u'no USB Connection to actiCAP'], 2 in total
Some event codes could not be mapped to integers. Use the `event_id` parameter to map such events to integers manually.
<Info | 16 non-empty fields
    bads : list | 0 items
    buffer_size_sec : float | 1.0
    ch_names : 'mne.io.meas_info._ChannelNameList | Fp1, Fp2, F7, F3, Fz, F4, F8, FC5, FC1, FC2
    chs : list | 33 items (EEG: 32, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : 'mne.transforms.Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 250.0 Hz
    meas_date : numpy.ndarray | 1970-01-01 02:00:00
    nchan : int | 33
    projs : list | 0 items
    sfreq : float | 500.0 Hz
    acq_pars

/usr/lib64/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  app.launch_new_instance()
/usr/lib64/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: Events like the following will be dropped entirely: [u'boundary', u'no USB Connection to actiCAP'], 2 in total
  app.launch_new_instance()
/usr/lib64/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: Some event codes could not be mapped to integers. Use the `event_id` parameter to map such events to integers manually.
  app.launch_new_instance()


In [ ]:
print((raw.ch_names))

In [12]:
events = mne.find_events(raw)

In [13]:
print (events)

[[  36118       0       2]
 [1613461       0       2]]


In [14]:
# Pick the centro-parietal channels
raw.preload = True
centro_parietal_channels = ['C3','Cz','C4','CP1','CP2','CP5','CP6','P3','Pz','P4']
specific_chans = raw.pick_channels(centro_parietal_channels, copy=True)
print(specific_chans)

<RawEEGLAB  |  matti_rs_cl_fl.fdt, n_channels x n_times : 10 x 1651660 (3303.3 sec)>


In [15]:
first_event_sec = specific_chans.times[36118]
last_event_sec = specific_chans.times[1613461]
print("Experiment times %f --> %f" % (first_event_sec, last_event_sec))

Experiment times 72.236000 --> 3226.922000


Experiment times 72.235800 --> 3226.920800

In [17]:
#last_event_sec = specific_chans.times[1490754]

#FIRST INDEX WORK AROUND
#pouyan_sec = 48 * 60 + 30.16

#first_event_sec = last_event_sec - pouyan_sec

#print("Total Pouyan time: %f" % (pouyan_sec))
#pouyan_start = indexes[0] - pouyanMs

#first_event_sec = 72.235800
#last_event_sec = 3226.920800


#start, stop = specific_chans.time_as_index([first_event_sec,last_event_sec]) 

start = events[0][0]
stop = events[1][0]
print(start,stop)


(36118, 1613461)


In [18]:
specific_chans = specific_chans.load_data()
specific_chans.preload = False
data, times = specific_chans[:, start:stop]
print(specific_chans)

<RawEEGLAB  |  matti_rs_cl_fl.fdt, n_channels x n_times : 10 x 1651660 (3303.3 sec)>


In [19]:
print(len(data[0]))

1577343


In [20]:
def get_lpp(index):    
    first_index = int(index + raw.info['sfreq'] * 0.4)
    last_index = int(index + raw.info['sfreq'] * 0.7)
    if last_index < len(data[0]):
        #print("Will get maximum value between index %d and %d" % (first_index, last_index))       
        
        max_amplitude = data[:,first_index:last_index + 1].sum(axis=0).max()
        present_value = data[:,index].sum()
        
        lpp  = max_amplitude - present_value
        return lpp
    else:
        return 0
        
lpp_values = []
for i in range(len(data[0])):
    lpp_values.append(get_lpp(i))

               

In [33]:
#plt.rcParams["figure.figsize"] = [20.0,8.0]
#%pylab qt
a = hist(lpp_values, bins=500)


Populating the interactive namespace from numpy and matplotlib


In [37]:
print(a[1][180])

0.000300010442151


In [38]:
conn = sqlite3.connect('matti_eeg.db')
c = conn.cursor()

In [39]:
# Create table
c.execute('''CREATE TABLE data
             (
             channel0 real, 
             channel1 real, 
             channel2 real, 
             channel3 real, 
             channel4 real, 
             channel5 real, 
             channel6 real, 
             channel7 real, 
             channel8 real, 
             channel9 real, 
             lpp real)''')

In [40]:
channels = []
for value_index in range(len(data[0])):
    all_channels = []
    for channel_index in range(10):
        all_channels.append(data[channel_index][value_index])
    all_channels.append(lpp_values[value_index])
        
    channels.append(tuple(all_channels))      
    
    
#channel0 = map(lambda x: (0,x), data[0])


In [41]:
#print(data[1][1])

In [42]:
c.executemany('INSERT INTO data VALUES (?,?,?,?,?,?,?,?,?,?,?)', channels)

In [43]:
conn.commit()
conn.close()